# Bandgap Prediction

Using QM9 data set to predict the bandgap of molecules using LSTM and GNN's

In [25]:
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader as GraphDataLoader
import torch
import itertools



SyntaxError: invalid syntax (3022400055.py, line 1)